# Service-Aware Google Colab Notebook

In this notebook, we load and pre-process the data collected from the network traffic scenarios. Moreover, we define, train and evaluate an LSTM model to fit the data optimally.
<br>
<br>

### Upload collected data

In [12]:
from google.colab import files
uploaded = files.upload()

Saving data_range_0_15.csv to data_range_0_15.csv
Saving train_data2_range0_15.csv to train_data2_range0_15.csv


### Mount to Google Drive

We mount the notebook to google drive to save our final model.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Import necessary libraries

In [6]:
import os
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from os import walk
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

from random import random
from random import randint
from numpy import array
from numpy import zeros
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed

from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import RepeatVector
from keras.layers import Bidirectional
from keras.layers import GRU

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# Pre-processing

Start with pre-processing the augmentation data.

In [13]:
train_df = pd.read_csv('train_data2_range0_15.csv')
train_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,13543.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021667,0.000000,0.0,12.0,15.0,15.0
1,7911.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021021,0.000000,0.0,14.0,15.0,14.0
2,26852.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025529,0.000000,0.0,15.0,15.0,15.0
3,37829.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022622,0.000000,0.0,12.0,14.0,14.0
4,51536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021853,0.000000,0.0,11.0,15.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48595,0.0,0.0,0.0,2716.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016907,0.0,13.0,15.0,10.0
48596,0.0,0.0,0.0,3531.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.018335,0.0,14.0,13.0,7.0
48597,0.0,0.0,0.0,5497.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016292,0.0,14.0,15.0,9.0
48598,0.0,0.0,0.0,2382.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017230,0.0,14.0,15.0,12.0


In [15]:
test_df = pd.read_csv('data_range_0_15.csv')
test_df = test_df.drop(['Time'],axis=1)
test_df = test_df.drop(['empty'],axis=1)
test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,15283,0,0,0,0,0,0,0,0,0.018868,0.000000,0.0,13,14,15
1,8248,0,0,0,0,0,0,0,0,0.021510,0.000000,0.0,13,14,15
2,27504,0,0,0,0,0,0,0,0,0.023119,0.000000,0.0,13,15,15
3,38723,0,0,0,0,0,0,0,0,0.022645,0.000000,0.0,13,14,15
4,51684,0,0,0,0,0,0,0,0,0.021363,0.000000,0.0,13,14,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0,0,0,3202,0,0,0,0,0,0.000000,0.016926,0.0,13,15,9
158,0,0,0,3202,0,0,0,0,0,0.000000,0.017017,0.0,13,15,10
159,0,0,0,3202,0,0,0,0,0,0.000000,0.017106,0.0,13,15,10
160,0,0,0,3202,0,0,0,0,0,0.000000,0.017195,0.0,13,15,11


Replace None Values

In [ ]:
# train_df = train_df.replace('None', 0)
# test_df = test_df.replace('None', 0)

### Define Pre-processing Functions

In [16]:
def extract_X_y_from_file(array, X_num, y_num):
    """ 
    Function that pre-process the data with a sliding-window supervised 
    learning approach extracting X and y

    - X_num: Input Window Length (N)
    - y_num: Number of prediction time-steps (M). We calculate the y as a mean 
              of the next M time-slots.
    """

    X = []
    y = []
    
    iterations = len(array)
    for i in range(iterations):
        ### check if we reach to end
        if i+X_num + y_num > len(array):
            break
            
        ### calculate X,y
        X_temp = array[i: i+X_num]
        y_temp = array[i+X_num: i+X_num + y_num]
        
        ### format y use mean
        new_y_temp = []
        # for every column
        for j in range(15):
            j_temp = []
            # for every row
            for i in range(y_num):
                row = y_temp[i][j]                

                j_temp.append(row)

            # obtain mean
            new_y_temp.append(np.mean(j_temp))
                        
        
        # append
        X.append(X_temp)
        y.append(new_y_temp)        
        
    return X,y

In [17]:
def create_data(df, X_num = 30, y_num = 5):
    """
    Function that reads all data csvs and calls the above mentioned function to
    extract X and y
    """
    all_X = []
    all_y = []

    # convert to list
    array = df.values.tolist()

    # create X and y
    X,y = extract_X_y_from_file(array, X_num, y_num)

    # add to final data
    all_X = all_X + X
    all_y = all_y + y

    return all_X, all_y

Convert object columns to float

In [18]:
train_df.dtypes

UE1: web-rtc       float64
UE1: sipp          float64
UE1: web-server    float64
UE2: web-rtc       float64
UE2: sipp          float64
UE2: web-server    float64
UE3: web-rtc       float64
UE3: sipp          float64
UE3: web-server    float64
UE1-Jitter         float64
UE2-Jitter         float64
UE3-Jitter         float64
UE1-CQI            float64
UE2-CQI            float64
UE3-CQI            float64
dtype: object

In [19]:
train_df['UE1-Jitter'] = train_df['UE1-Jitter'].astype(float)
train_df['UE2-Jitter'] = train_df['UE2-Jitter'].astype(float)
train_df['UE3-Jitter'] = train_df['UE3-Jitter'].astype(float)
train_df.dtypes

UE1: web-rtc       float64
UE1: sipp          float64
UE1: web-server    float64
UE2: web-rtc       float64
UE2: sipp          float64
UE2: web-server    float64
UE3: web-rtc       float64
UE3: sipp          float64
UE3: web-server    float64
UE1-Jitter         float64
UE2-Jitter         float64
UE3-Jitter         float64
UE1-CQI            float64
UE2-CQI            float64
UE3-CQI            float64
dtype: object

In [20]:
test_df['UE1-Jitter'] = test_df['UE1-Jitter'].astype(float)
test_df['UE2-Jitter'] = test_df['UE2-Jitter'].astype(float)
test_df['UE3-Jitter'] = test_df['UE3-Jitter'].astype(float)
test_df.dtypes

UE1: web-rtc         int64
UE1: sipp            int64
UE1: web-server      int64
UE2: web-rtc         int64
UE2: sipp            int64
UE2: web-server      int64
UE3: web-rtc         int64
UE3: sipp            int64
UE3: web-server      int64
UE1-Jitter         float64
UE2-Jitter         float64
UE3-Jitter         float64
UE1-CQI              int64
UE2-CQI              int64
UE3-CQI              int64
dtype: object

In [21]:
train_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,13543.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021667,0.000000,0.0,12.0,15.0,15.0
1,7911.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021021,0.000000,0.0,14.0,15.0,14.0
2,26852.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025529,0.000000,0.0,15.0,15.0,15.0
3,37829.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022622,0.000000,0.0,12.0,14.0,14.0
4,51536.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021853,0.000000,0.0,11.0,15.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48595,0.0,0.0,0.0,2716.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016907,0.0,13.0,15.0,10.0
48596,0.0,0.0,0.0,3531.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.018335,0.0,14.0,13.0,7.0
48597,0.0,0.0,0.0,5497.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016292,0.0,14.0,15.0,9.0
48598,0.0,0.0,0.0,2382.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017230,0.0,14.0,15.0,12.0


# Normalize Data

In [22]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
train_df_normalized = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns)

In [23]:
train_df_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,0.234195,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.568459,0.000000,0.0,0.785714,1.000000,1.000000
1,0.136802,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.551499,0.000000,0.0,0.928571,1.000000,0.909091
2,0.464343,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.669791,0.000000,0.0,1.000000,1.000000,1.000000
3,0.654164,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.593502,0.000000,0.0,0.785714,0.916667,0.909091
4,0.891195,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.573347,0.000000,0.0,0.714286,1.000000,0.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48595,0.000000,0.0,0.0,0.031921,0.0,0.0,0.0,0.0,0.0,0.000000,0.757520,0.0,0.857143,1.000000,0.545455
48596,0.000000,0.0,0.0,0.041500,0.0,0.0,0.0,0.0,0.0,0.000000,0.821486,0.0,0.928571,0.833333,0.272727
48597,0.000000,0.0,0.0,0.064606,0.0,0.0,0.0,0.0,0.0,0.000000,0.729940,0.0,0.928571,1.000000,0.454545
48598,0.000000,0.0,0.0,0.027996,0.0,0.0,0.0,0.0,0.0,0.000000,0.771971,0.0,0.928571,1.000000,0.727273


In [24]:
test_df_normalized = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)  

In [25]:
test_df_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,0.264284,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.495035,0.000000,0.0,0.857143,0.916667,1.000000
1,0.142630,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.564329,0.000000,0.0,0.857143,0.916667,1.000000
2,0.475617,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.606560,0.000000,0.0,0.857143,1.000000,1.000000
3,0.669624,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.594122,0.000000,0.0,0.857143,0.916667,1.000000
4,0.893754,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.560491,0.000000,0.0,0.857143,0.916667,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.000000,0.0,0.0,0.037633,0.0,0.0,0.0,0.0,0.0,0.000000,0.758388,0.0,0.857143,1.000000,0.454545
158,0.000000,0.0,0.0,0.037633,0.0,0.0,0.0,0.0,0.0,0.000000,0.762431,0.0,0.857143,1.000000,0.545455
159,0.000000,0.0,0.0,0.037633,0.0,0.0,0.0,0.0,0.0,0.000000,0.766435,0.0,0.857143,1.000000,0.545455
160,0.000000,0.0,0.0,0.037633,0.0,0.0,0.0,0.0,0.0,0.000000,0.770412,0.0,0.857143,1.000000,0.636364


# Sliding-Window Approach 
Create windows of data with the sliding-window technique to construct a supervised learning problem.

In [26]:
all_X, all_y = create_data(df=train_df_normalized)

In [27]:
len(all_X)

48566

In [28]:
len(all_y)

48566

In [29]:
X_data = np.array(all_X)
X_data.shape

(48566, 30, 15)

In [30]:
y_data = np.array(all_y)
print(y_data.shape)

(48566, 15)


Pre-process the basic non-augmented scenario to use as validation data.

In [31]:
val_X = []
val_y = []

val_X, val_y = create_data(df=test_df_normalized)

X_val = np.array(val_X)
print(X_val.shape)

y_val = np.array(val_y)
print(y_val.shape)

(128, 30, 15)
(128, 15)


In [32]:
print(X_data.shape)
print(y_data.shape)
print(X_val.shape)
print(y_val.shape)

(48566, 30, 15)
(48566, 15)
(128, 30, 15)
(128, 15)


# CNN-LSTM Model Implementation

In [33]:
# define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(30,15)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(25, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(15))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 29, 64)            1984      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 repeat_vector (RepeatVector  (None, 1, 896)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 1, 25)             92200     
                                                                 
 lstm_1 (LSTM)               (None, 25)                5

Define a callback function to save on every iteration the best optimal trained model, based on validation accuracy. Then fit the model to the data.

In [34]:
from keras.callbacks import ModelCheckpoint
filepath="/content/gdrive/MyDrive/v2__app_aware_CNN-LSTM_15cols__train_data2_range0_15__test_datarange0-15.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [35]:
# save scaler
import joblib
scaler_filename = F"/content/gdrive/MyDrive/v2_train_scaler_app_aware_CNN-LSTM.save"
joblib.dump(scaler, scaler_filename)

['/content/gdrive/MyDrive/v2_train_scaler_app_aware_CNN-LSTM.save']

In [36]:
model.fit(X_data,y_data, validation_data=(X_val,y_val),epochs=100, callbacks=callbacks_list)
#model.fit(X_data,y_data,epochs=300)

Epoch 1/100
1515/1518 [============================>.] - ETA: 0s - loss: 0.0084
Epoch 1: val_loss improved from inf to 0.00076, saving model to /content/gdrive/MyDrive/v2__app_aware_CNN-LSTM_15cols__train_data2_range0_15__test_datarange0-15.best.hdf5
1518/1518 [==============================] - 16s 8ms/step - loss: 0.0084 - val_loss: 7.6230e-04
Epoch 2/100
1511/1518 [============================>.] - ETA: 0s - loss: 8.4773e-04
Epoch 2: val_loss improved from 0.00076 to 0.00063, saving model to /content/gdrive/MyDrive/v2__app_aware_CNN-LSTM_15cols__train_data2_range0_15__test_datarange0-15.best.hdf5
1518/1518 [==============================] - 11s 7ms/step - loss: 8.4721e-04 - val_loss: 6.2652e-04
Epoch 3/100
1512/1518 [============================>.] - ETA: 0s - loss: 6.8397e-04
Epoch 3: val_loss improved from 0.00063 to 0.00039, saving model to /content/gdrive/MyDrive/v2__app_aware_CNN-LSTM_15cols__train_data2_range0_15__test_datarange0-15.best.hdf5
1518/1518 [========================

Now the model is trained. Make a prediction test ( with 1 value, the X[0] ) to present the format of the input and the format of the predictions.

In [37]:
test = X_data[0].reshape(1,30,15)
test.shape

(1, 30, 15)

In [38]:
test

array([[[0.23419451, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.56845945,
         0.        , 0.        , 0.78571429, 1.        , 1.        ],
        [0.13680224, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.55149923,
         0.        , 0.        , 0.92857143, 1.        , 0.90909091],
        [0.46434253, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.66979096,
         0.        , 0.        , 1.        , 1.        , 1.        ],
        [0.65416407, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.5935021 ,
         0.        , 0.        , 0.78571429, 0.91666667, 0.90909091],
        [0.89119458, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.57334717,
         0.        , 0.       

In [39]:
yhat = model.predict(test)
yhat

array([[ 8.8256425e-01, -2.2646205e-05, -1.5440583e-03,  1.7668344e-03,
         7.9013789e-01, -8.6022419e-06, -1.7868504e-03, -1.2700168e-05,
         2.0935126e-03,  6.6549778e-01,  7.1051836e-01, -5.7013109e-03,
         6.8946999e-01,  7.8396434e-01,  9.4715786e-01]], dtype=float32)

In [40]:
y_data[0]

array([0.85870858, 0.        , 0.        , 0.        , 0.77836731,
       0.        , 0.        , 0.        , 0.        , 0.67694171,
       0.69464086, 0.        , 0.7       , 0.75      , 0.89090909])

# Inverse scaling to return in the original scale

In [47]:
y = y_data[0]
y.shape

(15,)

In [42]:
y = y.reshape(-1,1)
y.shape

(15, 1)

In [43]:
len(train_df.columns)

15

In [48]:
pd.Series(y)

0     0.858709
1     0.000000
2     0.000000
3     0.000000
4     0.778367
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.676942
10    0.694641
11    0.000000
12    0.700000
13    0.750000
14    0.890909
dtype: float64

In [49]:
single_test_df = pd.DataFrame(columns=train_df.columns)
single_test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI


In [50]:
single_test_df.loc[len(test_df)] = y

In [51]:
single_test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
162,0.858709,0.0,0.0,0.0,0.778367,0.0,0.0,0.0,0.0,0.676942,0.694641,0.0,0.7,0.75,0.890909


In [52]:
scaler.inverse_transform(single_test_df)

array([[4.96574000e+04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.66116000e+04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.58018705e-02, 1.55037248e-02, 0.00000000e+00,
        1.08000000e+01, 1.20000000e+01, 1.38000000e+01]])

In [53]:
# complete testing data frame

test_df_inv_normalized = pd.DataFrame(scaler.inverse_transform(test_df_normalized), columns=test_df_normalized.columns)  
test_df_inv_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,15283.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018868,0.000000,0.0,13.0,14.0,15.0
1,8248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021510,0.000000,0.0,13.0,14.0,15.0
2,27504.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023119,0.000000,0.0,13.0,15.0,15.0
3,38723.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022645,0.000000,0.0,13.0,14.0,15.0
4,51684.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021363,0.000000,0.0,13.0,14.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.0,0.0,0.0,3202.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.016926,0.0,13.0,15.0,9.0
158,0.0,0.0,0.0,3202.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017017,0.0,13.0,15.0,10.0
159,0.0,0.0,0.0,3202.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017106,0.0,13.0,15.0,10.0
160,0.0,0.0,0.0,3202.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017195,0.0,13.0,15.0,11.0


# Predictions - Evaluation

## Training Evaluation

We load the (same) model from the saved HDF5 file in our repository. We begin by evaluating the model on the training - augmented data.

In [54]:
from keras.models import load_model

In [55]:
model = load_model('/content/gdrive/MyDrive/v2__app_aware_CNN-LSTM_15cols__train_data2_range0_15__test_datarange0-15.best.hdf5')

In [56]:
X_data.shape

(48566, 30, 15)

In [57]:
yhat_training = model.predict(X_data)

In [58]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

rms = mean_squared_error(y_data, yhat_training)
print('RMSE training',rms)

mae = mean_absolute_error(y_data, yhat_training)
print('MAE: training',mae)

RMSE training 0.0003913822232830389
MAE: training 0.01117457622192524


In our algorithm we use the *round()* function to round the predictions.

In [59]:
print(yhat_training[100])
print(y_data[100])

[ 8.7897480e-04 -1.9085758e-04  5.2462524e-01  4.4405967e-01
  4.7078012e-03  2.9454884e-04 -4.7706142e-03  1.7468201e-04
  3.7229681e-01  4.3563712e-01  7.2175306e-01  5.3688401e-01
  8.4993947e-01  9.6304041e-01  4.9356127e-01]
[0.         0.         0.51820133 0.45694541 0.         0.
 0.         0.         0.39339247 0.46942363 0.72642675 0.54990626
 0.88571429 0.93333333 0.54545455]


In [60]:
for i in yhat_training[100]:
  print(round(i,2))

0.0
-0.0
0.52
0.44
0.0
0.0
-0.0
0.0
0.37
0.44
0.72
0.54
0.85
0.96
0.49


## Testing Evaluation

Now, use the validation X data (X-val) from the basic non-augmented scenario to make predictions. Then evaluate the model utilizing the MAE, and RMSE metrics. 

In [61]:
yhat_testing = model.predict(X_val)

In [62]:
rms = mean_squared_error(y_val, yhat_testing)
print('RMSE testing ',rms)

mae = mean_absolute_error(y_val, yhat_testing)
print('MAE: testing ',mae)

RMSE testing  0.00014026361061522305
MAE: testing  0.006002722580051458
